In [2]:
using DataFrames
using Query
using ProgressMeter
using Plots
pyplot()

include("spike.jl/NeuroAnalysis.jl")

NeuroAnalysis

In [40]:
_session = "ts13_V1_180514_COLOR2"
_dir = "data/3GratingSequence/SpikeTrain"
_chart = joinpath("data/3GratingSequence", "CHART", _session)
isdir(_chart)||mkdir(_chart)
#_csvpath = joinpath(_dir, "SpikeTrain", "$_session.csv")
#_matpath = joinpath(_dir, "S

In [50]:
function block_chunker(_table, _train)
    _chunked = Dict(
        "green"=>[60.0*(i-1) + _train[3] for i=1:14],
        "blue" =>[60.0*(i-1) + _train[3] + 30.0 for i=1:14]
    )
    
end

@time _spike_units, _spike_marker = NeuroAnalysis.SpikeUnits.import_spike_train_data(
    _session, "1",
    _dir = _dir,
    _chunker=block_chunker,
    _marker_channel="Text_Mark"
);

  0.243740 seconds (56.38 k allocations: 4.756 MiB, 12.60% gc time)


In [51]:
_spike_marker.chunked_marker

Dict{String,Array{Float64,1}} with 2 entries:
  "blue"  => [119.602, 179.602, 239.602, 299.602, 359.602, 419.602, 479.602, 53…
  "green" => [89.6016, 149.602, 209.602, 269.602, 329.602, 389.602, 449.602, 50…

In [47]:
14*60+90

930

In [45]:
_spike_units["CSC6a"]

NeuroAnalysis.SpikeUnits.SpikeUnit("ts13_V1_180514_COLOR2", "1", "CSC6a", [0.04155, 0.0448, 0.0522, 60.8675, 61.0062, 61.0165, 61.0268, 61.1051, 61.1127, 61.126  …  979.731, 979.769, 979.856, 979.866, 979.873, 979.875, 979.895, 979.92, 979.949, 979.978])

In [27]:
nord = Dict(
    :polarnight => ("#2e3440", "#3b4252", "#434c5e", "#4c566a"),            # polar night
    :snowstorm  => ("#d8dee9", "#e5e9f0", "#eceff4"),                       # snow storm
    :frost      => ("#8fbcbb", "#88c0d0", "#81a1c1", "#5e81ac"),            # frosta
    :aurora     => ("#bf616a", "#d08770", "#ebcb8b", "#a3be8c", "#b48ead")  # aurora
);

In [33]:
roi = -5:0.1:25
backgrounds = -4:0.1:-1
pbar = Progress(length(_spike_units), dt=1)
for (channel, spike) in _spike_units
    _train = spike.spike_train
    k = NeuroAnalysis.LinearFilter.gaussian_kernel(_train, 0.4);
    dx = NeuroAnalysis.LinearFilter.apply_roi_aggregate(k, _spike_marker.chunked_marker["ABABA"], roi, backgrounds);
    
    
    #===========plotting===========#
    ABABA_color = nord[:aurora][5]
    ABABB_color = nord[:aurora][4]

    summary = plot(roi, mean(dx,2), ribbon=std(dx,2), fillalpha=0.1, linewidth=2, color=ABABA_color, legend=false);

    plot(roi, mean(dx,2),fillalpha=0.1,linewidth=4, color=ABABA_color, legend=false);
    ABABA = plot!(roi, dx, alpha=0.1, color=ABABA_color, legend=false);
    plot(summary,ABABA,layout=(2,1), size=(900,600))
    savefig(joinpath(_chart, "$(_session)_$(channel).png"))
    
    next!(pbar)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:24


---
## color block

In [53]:
roi = -5:0.1:25
backgrounds = -4:0.1:-1
pbar = Progress(length(_spike_units), dt=1)
for (channel, spike) in _spike_units
    _train = spike.spike_train
    k = NeuroAnalysis.LinearFilter.gaussian_kernel(_train, 0.4);
    green = NeuroAnalysis.LinearFilter.apply_roi_aggregate(k, _spike_marker.chunked_marker["green"], roi, backgrounds);
    blue  = NeuroAnalysis.LinearFilter.apply_roi_aggregate(k, _spike_marker.chunked_marker["blue"], roi, backgrounds);
    
    
    #===========plotting===========#
    blue_color = nord[:frost][4]
    green_color = nord[:aurora][4]

    plot(roi, mean(green,2), ribbon=std(green,2), fillalpha=0.1, linewidth=2, color=green_color, legend=false);
    summary = plot!(roi, mean(blue,2), ribbon=std(blue,2), fillalpha=0.1, linewidth=2, color=blue_color, legend=false);
    
    plot(roi, mean(green,2),fillalpha=0.1,linewidth=4, color=green_color, legend=false);
    green_plot = plot!(roi, green, alpha=0.1, color=green_color, legend=false);
    
    plot(roi, mean(blue,2),fillalpha=0.1,linewidth=4, color=blue_color, legend=false);
    blue_plot = plot!(roi, blue, alpha=0.1, color=blue_color, legend=false);
    
    plot(summary,green_plot,blue_plot,layout=(3,1), size=(900,900))
    savefig(joinpath(_chart, "$(_session)_$(channel).png"))
    
    next!(pbar)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:52
